In [ ]:
#importing libraries

import random as rd
import hashlib
import phe
import rsa

In [ ]:
#declaring of varibles
numberOfExperts = 3
numberOfCandidates = 50
#collection of homomophic public and private keys
dictionaryOfExpertPublicKeys = {}
dictionaryOfExpertPrivateKeys = {}

dictionaryOfCandidatesPublicKeys = {}
dictionaryOfCandidatesPrivateKeys = {}

#Variables for third framework@
dictionaryOfRandomNumberPerExpert = {}
dictionaryOftheSumOfRandomNumbermappedToEveryCandidate = {}
dictionaryOfRandomNumberGatheredFromOtherExperts = {}
dictionaryOfRandomNumberMappedToEveryCandidate = {}

#Collection of rsa public and private keys for digital signature
dictOfrsaPubKey = {}
dictOfrsaPriKey = {}
marksToCandidates = [[0 for i in range(numberOfCandidates)] for j in range(numberOfExperts)]


In [ ]:
#this part of the code create the public and private homomorphic key pairs

for i in range(numberOfExperts):
    x,y = phe.generate_paillier_keypair(n_length=128)
    dictionaryOfExpertPublicKeys[i] = x
    dictionaryOfExpertPrivateKeys[i] = y
    

for i in range(numberOfCandidates):
    x,y = phe.generate_paillier_keypair(n_length=128)
    dictionaryOfCandidatesPublicKeys[i]= x
    dictionaryOfCandidatesPrivateKeys[i] = y 

publicKeyEmployer,privateKeyEmployer = phe.generate_paillier_keypair(n_length=128)


In [ ]:
#@code to generate rsa public and private keys
for i in range(numberOfExperts):
    (pubkey, privkey) = rsa.newkeys(512)
    dictOfrsaPubKey[i],dictOfrsaPriKey[i] = pubkey, privkey 


In [ ]:
# function to create digital signature
def createDigitalSignature(message,i):
    privkey = dictOfrsaPriKey[i]
    temp = str(message).encode()
    signature = rsa.sign(temp, privkey, 'SHA-1')
    return signature

In [ ]:
def generateRandomNumberForEveryExpert():
    # global dictionaryOfRandomNumberPerExpert
    generatedRandomNumbers = [rd.randint(0,100) for x in range(numberOfExperts)]
    # print(generatedRandomNumbers)
    return generatedRandomNumbers
    # print(dictionaryOfSumGeneratedPerRandomNumber)

In [ ]:
def forwardTheRespectiveRandomNumbers(idR,generatedRandomNumbers):
    encryptedRandomNumber = dictionaryOfExpertPublicKeys[idR].raw_encrypt(generatedRandomNumbers[idR])
    message = encryptedRandomNumber
    return message
# ,createDigitalSignature(encryptedRandomNumber,idS)

In [ ]:
def performSMPCBetweenExperts():
    #this part of the code simulates the part where expert follow the smpc process
    #the random number assgned by all the experts has been mapped for every candidate
    
    for i in range(numberOfCandidates):
        matrixOfForwardedMessages = [[0 for i in range(numberOfExperts)] for j in range(numberOfExperts)]
        sumList = []
        for j in range(numberOfExperts):
            generatedRandomNumbers = generateRandomNumberForEveryExpert()
            sumOfTheRandomNumbers = sum(generatedRandomNumbers) 
            sumList.append(sumOfTheRandomNumbers)

            for k in range(numberOfExperts):
                if(j != k):
                    message = forwardTheRespectiveRandomNumbers(k,generatedRandomNumbers)
                    matrixOfForwardedMessages[j][k] = message
                else:
                    matrixOfForwardedMessages[j][k] = generatedRandomNumbers[j]
        dictionaryOfRandomNumberMappedToEveryCandidate[i] = matrixOfForwardedMessages
        dictionaryOftheSumOfRandomNumbermappedToEveryCandidate[i] = sumList
    print(dictionaryOfRandomNumberMappedToEveryCandidate)
    print(dictionaryOftheSumOfRandomNumbermappedToEveryCandidate)
    # print(dictionaryOfSumGeneratedPerRandomNumber)
    

    

In [ ]:
def generateHiddenScoreFrameWork4():
    global marksToCandidates
    for i in range(numberOfCandidates):
        for j in range(numberOfExperts):
            mark = rd.randint(0,100)
            hiddenScore = mark + dictionaryOftheSumOfRandomNumbermappedToEveryCandidate[i][j]
            sum  = 0
            for k in range(numberOfExperts):
                if(j != k):
                    encryptedRandomNumber = dictionaryOfRandomNumberMappedToEveryCandidate[i][k][j]
                    # print(encryptedRandomNumber)
                    sum  = sum + dictionaryOfExpertPrivateKeys[j].raw_decrypt(encryptedRandomNumber)
                else: 
                    sum = sum + dictionaryOfRandomNumberMappedToEveryCandidate[i][j][j]
            hiddenScore  = hiddenScore -sum
            # print(hiddenScore)
            marksToCandidates[j][i] = publicKeyEmployer.raw_encrypt(hiddenScore)

In [ ]:
def employerFindsTheBestCandidate(array):
    marksOfCandidates = [privateKeyEmployer.raw_decrypt(x) for x in array]
    print(marksOfCandidates)
    print(marksOfCandidates.index(max(marksOfCandidates)))


In [ ]:
def candidateManagerFrameWork4BestCandidate():
    array = []
    for i in range(numberOfCandidates):
        sum = 1
        for j in range(numberOfExperts):
            cummulativeScore = marksToCandidates[j][i]
            sum  = sum * cummulativeScore
        array.append(sum)
    employerFindsTheBestCandidate(array)


In [ ]:
def framework4():
    performSMPCBetweenExperts()
    generateHiddenScoreFrameWork4()
    candidateManagerFrameWork4BestCandidate()


In [ ]:
framework4()